# Feature selection and data splitting for mds data 

In [1]:
import numpy as np
import polars as pl
from sklearn.model_selection import StratifiedKFold

In [2]:
y = pl.read_csv("mds_data/preprocessed/annotations_66.csv")

y = (
    y.select("SAMPLE_ID", "disease")
    .rename({"SAMPLE_ID": "sample_ids", "disease": "class"})
    .with_columns(pl.col("class") - 1)
)
y

sample_ids,class
str,i64
"""N58""",0
"""N60""",0
"""N70""",0
"""N82""",0
"""N83""",0
…,…
"""V806""",1
"""V839""",1
"""V883""",1


In [3]:
mrna = pl.read_csv("mds_data/preprocessed/mrna.csv")
mrna

GENE_ID,GENE_NAME,N58,N60,N70,N82,N83,N84,N85,V1048,V108,V1090,V125,V1279,V1297,V1321,V1337,V1394,V1441,V1456,V148,V1505,V1528,V1565,V1591,V1592,V1699,V1708,V1742,V1776,V1788,V18,V1800,V1823,V1834,V1857,V1860,V1874,V1884,V1920,V1921,V2089,V2092,V2110,V2133,V221,V344,V359,V406,V456,V513,V538,V553,V574,V624,V630,V637,V655,V67,V712,V714,V716,V777,V806,V839,V883,V888,V940
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""ENSG00000103184""","""SEC14L5""",4.574155,7.288239,3.561246,2.897224,3.615696,4.057362,2.203009,4.398603,3.467058,3.657675,4.619424,1.507977,4.809069,4.749144,2.146183,2.66225,3.555635,0.714897,2.010324,6.268556,5.914118,4.018251,6.781959,4.900504,3.481129,3.767197,5.167452,1.631362,5.933825,4.945851,0.444571,2.734007,4.452827,5.877369,6.07671,3.343197,6.156233,2.993291,5.727105,1.36271,5.001145,5.516594,6.915722,3.82929,3.319966,6.938185,5.118215,1.629126,5.885573,5.003287,2.039422,10.53412,2.297346,4.097657,3.891976,4.604959,5.844351,3.188559,1.986391,2.357737,3.549326,5.252131,3.976603,5.670529,4.456624,4.897539
"""ENSG00000119862""","""LGALSL""",7.455039,8.479401,6.306391,3.321911,6.188541,5.979059,5.223318,6.5576,6.140982,7.953992,6.599124,5.548295,8.013039,7.647353,4.956439,6.928581,5.090716,6.687677,5.006865,9.311978,8.342994,6.413821,9.084263,6.776719,5.304247,6.598114,6.798145,4.599317,8.505406,6.422409,2.381453,6.143426,5.936016,8.052438,7.556633,6.799104,7.551962,6.284811,7.092467,3.705927,6.91619,6.314894,6.214188,5.860457,5.514356,8.538408,7.514117,6.082184,9.708378,6.616514,4.57679,12.548,5.836737,6.590203,6.043704,7.598248,7.729715,6.108103,5.62265,5.729892,5.977239,6.672379,6.838144,7.450425,6.055094,6.88753
"""ENSG00000115461""","""IGFBP5""",1.531313,4.93984,1.589817,3.596918,4.221713,5.694364,4.633449,7.55643,2.906119,6.264272,4.300236,4.642322,5.192593,5.82839,3.354076,6.16894,3.134675,4.718852,8.624212,3.38863,3.416752,8.240399,6.652194,7.699936,4.466548,3.895035,8.655896,4.393156,3.639881,5.553176,2.653826,6.227905,6.201601,9.962816,5.073338,4.806445,4.322065,3.59729,4.244579,4.54983,5.835025,6.82944,7.039622,4.689163,3.702583,6.163571,2.613793,2.308573,3.431658,4.664411,3.645008,4.07661,4.614307,5.113326,2.241138,3.67498,11.902128,2.718183,2.048483,1.477194,4.562139,6.748317,2.710774,4.358541,5.20171,6.416449
"""ENSG00000137801""","""THBS1""",10.787846,12.452547,9.390428,7.256898,9.546596,9.807684,8.395226,10.564428,9.994399,10.93104,10.041629,8.696442,11.235067,10.286238,7.73148,9.966976,10.189488,11.291271,8.163448,12.26609,11.428713,9.155845,12.369392,8.678094,8.537479,11.617824,10.461017,7.391252,11.719449,8.622247,6.537787,9.79737,9.035041,11.235494,11.164126,9.850056,10.290641,7.654049,11.137592,4.781831,10.478594,10.905235,9.504674,9.022241,8.020058,12.340003,9.573093,9.417571,11.567134,10.372583,7.060916,15.367104,8.483557,10.596553,8.780714,9.818366,11.521034,6.721186,6.61051,9.027454,7.719915,10.210917,10.274893,10.805033,10.464737,10.704895
"""ENSG00000173083""","""HPSE""",6.935416,7.987946,7.20935,5.125137,6.211795,5.990862,5.462574,6.413457,7.825867,8.089623,6.922915,6.288086,7.527803,6.888642,6.086392,6.349528,6.562359,6.139168,6.209023,8.389047,7.645464,6.792004,8.256308,6.635347,6.015047,6.27286,6.943379,5.713779,8.075317,5.712303,5.481236,6.143426,6.635867,6.400915,7.285251,6.45702,8.3526,5.8538,7.281664,5.741307,6.923299,7.189791,7.09444,6.704163,4.665372,8.178149,6.607652,6.537152,7.83118,7.217909,6.983234,11.844895,5.8875,8.039698,6.089552,6.931796,7.643238,5.951342,5.730754,6.129718,7.313044,6.989801,7.451146,6.940394,6.860065,6.735584
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ENSG00000100320""","""RBFOX2""",8.22931,8.577745,8.793937,8.146168,8.50636,8.094

In [ ]:
from src.data_preprocessing import OmicDataSplitter

ods = OmicDataSplitter(
    df=mrna,
    annotation_cols=["GENE_ID", "GENE_NAME"],
    y_df=y,
    n_splits=5,
    random_state=3,
    output_dir="mds_data/splits/mrna",
)
ods.process_data()

In [17]:
A = np.eye(3)
A[0,0] = 2
A[0,1] = 3
print(A)
A.max(axis=0)

[[2. 3. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


array([2., 3., 1.])

In [21]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=3)

list(skf.split(np.zeros(len(y)), y["class"]))

[(array([ 0,  1,  2,  3,  6,  8,  9, 10, 13, 14, 15, 16, 17, 18, 19, 20, 21,
         23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35, 37, 38, 39, 40, 41, 43,
         45, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63,
         64]),
  array([ 4,  5,  7, 11, 12, 22, 24, 34, 36, 42, 44, 46, 62, 65])),
 (array([ 0,  1,  2,  4,  5,  7,  8, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21,
         22, 23, 24, 25, 26, 27, 28, 29, 30, 33, 34, 35, 36, 37, 38, 39, 40,
         41, 42, 43, 44, 45, 46, 48, 50, 52, 53, 54, 56, 57, 58, 59, 60, 61,
         62, 65]),
  array([ 3,  6,  9, 10, 15, 31, 32, 47, 49, 51, 55, 63, 64])),
 (array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 18,
         22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 40, 42, 43,
         44, 45, 46, 47, 48, 49, 51, 52, 54, 55, 57, 58, 59, 60, 61, 62, 63,
         64, 65]),
  array([ 0, 17, 19, 20, 21, 23, 37, 38, 39, 41, 50, 53, 56])),
 (array([ 0,  1,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 1

In [29]:
from src.data_splitting import OmicDataSplitter

ods = OmicDataSplitter(y, n_splits=5, random_state=3)

In [28]:
mrna = pl.read_csv("mds_data/preprocessed/mrna.csv")
mrna

GENE_ID,GENE_NAME,N58,N60,N70,N82,N83,N84,N85,V1048,V108,V1090,V125,V1279,V1297,V1321,V1337,V1394,V1441,V1456,V148,V1505,V1528,V1565,V1591,V1592,V1699,V1708,V1742,V1776,V1788,V18,V1800,V1823,V1834,V1857,V1860,V1874,V1884,V1920,V1921,V2089,V2092,V2110,V2133,V221,V344,V359,V406,V456,V513,V538,V553,V574,V624,V630,V637,V655,V67,V712,V714,V716,V777,V806,V839,V883,V888,V940
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""ENSG00000103184""","""SEC14L5""",4.574155,7.288239,3.561246,2.897224,3.615696,4.057362,2.203009,4.398603,3.467058,3.657675,4.619424,1.507977,4.809069,4.749144,2.146183,2.66225,3.555635,0.714897,2.010324,6.268556,5.914118,4.018251,6.781959,4.900504,3.481129,3.767197,5.167452,1.631362,5.933825,4.945851,0.444571,2.734007,4.452827,5.877369,6.07671,3.343197,6.156233,2.993291,5.727105,1.36271,5.001145,5.516594,6.915722,3.82929,3.319966,6.938185,5.118215,1.629126,5.885573,5.003287,2.039422,10.53412,2.297346,4.097657,3.891976,4.604959,5.844351,3.188559,1.986391,2.357737,3.549326,5.252131,3.976603,5.670529,4.456624,4.897539
"""ENSG00000119862""","""LGALSL""",7.455039,8.479401,6.306391,3.321911,6.188541,5.979059,5.223318,6.5576,6.140982,7.953992,6.599124,5.548295,8.013039,7.647353,4.956439,6.928581,5.090716,6.687677,5.006865,9.311978,8.342994,6.413821,9.084263,6.776719,5.304247,6.598114,6.798145,4.599317,8.505406,6.422409,2.381453,6.143426,5.936016,8.052438,7.556633,6.799104,7.551962,6.284811,7.092467,3.705927,6.91619,6.314894,6.214188,5.860457,5.514356,8.538408,7.514117,6.082184,9.708378,6.616514,4.57679,12.548,5.836737,6.590203,6.043704,7.598248,7.729715,6.108103,5.62265,5.729892,5.977239,6.672379,6.838144,7.450425,6.055094,6.88753
"""ENSG00000115461""","""IGFBP5""",1.531313,4.93984,1.589817,3.596918,4.221713,5.694364,4.633449,7.55643,2.906119,6.264272,4.300236,4.642322,5.192593,5.82839,3.354076,6.16894,3.134675,4.718852,8.624212,3.38863,3.416752,8.240399,6.652194,7.699936,4.466548,3.895035,8.655896,4.393156,3.639881,5.553176,2.653826,6.227905,6.201601,9.962816,5.073338,4.806445,4.322065,3.59729,4.244579,4.54983,5.835025,6.82944,7.039622,4.689163,3.702583,6.163571,2.613793,2.308573,3.431658,4.664411,3.645008,4.07661,4.614307,5.113326,2.241138,3.67498,11.902128,2.718183,2.048483,1.477194,4.562139,6.748317,2.710774,4.358541,5.20171,6.416449
"""ENSG00000137801""","""THBS1""",10.787846,12.452547,9.390428,7.256898,9.546596,9.807684,8.395226,10.564428,9.994399,10.93104,10.041629,8.696442,11.235067,10.286238,7.73148,9.966976,10.189488,11.291271,8.163448,12.26609,11.428713,9.155845,12.369392,8.678094,8.537479,11.617824,10.461017,7.391252,11.719449,8.622247,6.537787,9.79737,9.035041,11.235494,11.164126,9.850056,10.290641,7.654049,11.137592,4.781831,10.478594,10.905235,9.504674,9.022241,8.020058,12.340003,9.573093,9.417571,11.567134,10.372583,7.060916,15.367104,8.483557,10.596553,8.780714,9.818366,11.521034,6.721186,6.61051,9.027454,7.719915,10.210917,10.274893,10.805033,10.464737,10.704895
"""ENSG00000173083""","""HPSE""",6.935416,7.987946,7.20935,5.125137,6.211795,5.990862,5.462574,6.413457,7.825867,8.089623,6.922915,6.288086,7.527803,6.888642,6.086392,6.349528,6.562359,6.139168,6.209023,8.389047,7.645464,6.792004,8.256308,6.635347,6.015047,6.27286,6.943379,5.713779,8.075317,5.712303,5.481236,6.143426,6.635867,6.400915,7.285251,6.45702,8.3526,5.8538,7.281664,5.741307,6.923299,7.189791,7.09444,6.704163,4.665372,8.178149,6.607652,6.537152,7.83118,7.217909,6.983234,11.844895,5.8875,8.039698,6.089552,6.931796,7.643238,5.951342,5.730754,6.129718,7.313044,6.989801,7.451146,6.940394,6.860065,6.735584
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ENSG00000100320""","""RBFOX2""",8.22931,8.577745,8.793937,8.146168,8.50636,8.094

In [30]:
ods.process_data(
    mrna,
    "mds_data/splits/mrna",
    annotation_cols=["GENE_ID", "GENE_NAME"],
    normalization="minmax",
    feature_selection="mrmr",
    n_features=50,
)

TypeError: DataFrame.__init__() got an unexpected keyword argument 'columns'